[Source](https://beckernick.github.io/matrix-factorization-recommender/)


Importing stuff

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds

Mounting drive: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
ratings = pd.read_csv("/content/drive/MyDrive/Recommendation system project /ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
movies = pd.read_csv("/content/drive/MyDrive/Recommendation system project /movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
#merging the user and movie information and removing extra fields which we are not using 
ratings1 = pd.merge(movies,ratings)
ratings1=ratings1.drop(['genres','timestamp'],axis=1)
ratings1.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [ ]:
#Making user-movie rating matrix
matrix = ratings.pivot_table(index='userId',columns='movieId',values='rating') 
matrix.shape

(610, 9724)

In [ ]:
#Removing movies with very few ratings (10 is threshold) 
# matrix=matrix.dropna(thresh=10,axis=1)
# matrix.shape

In [ ]:
#converting NANs to 0
matrix=matrix.fillna(0)
matrix.head()
matrix_cpy=matrix

In [ ]:
#De-mean the data (normalize by each users mean) and convert it from a dataframe to a numpy array
matrix_cpy = matrix_cpy.to_numpy()
user_ratings_mean = np.mean(matrix_cpy, axis = 1)
R_normalized = matrix - user_ratings_mean.reshape(-1, 1)
R_normalized

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,3.895825,-0.104175,3.895825,-0.104175,-0.104175,3.895825,-0.104175,-0.104175,-0.104175,-0.104175,...,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175
2,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775,...,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775,-0.011775
3,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770,...,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770,-0.009770
4,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980,...,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980,-0.078980
5,3.983546,-0.016454,-0.016454,-0.016454,-0.016454,-0.016454,-0.016454,-0.016454,-0.016454,-0.016454,...,-0.016454,-0.016454,-0.016454,-0.016454,-0.016454,-0.016454,-0.016454,-0.016454,-0.016454,-0.016454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.080625,-0.419375,-0.419375,-0.419375,-0.419375,-0.419375,2.080625,-0.419375,-0.419375,-0.419375,...,-0.419375,-0.419375,-0.419375,-0.419375,-0.419375,-0.419375,-0.419375,-0.419375,-0.419375,-0.419375
607,3.927190,-0.072810,-0.072810,-0.072810,-0.072810,-0.072810,-0.072810,-0.072810,-0.072810,-0.072810,...,-0.072810,-0.072810,-0.072810,-0.072810,-0.072810,-0.072810,-0.072810,-0.072810,-0.072810,-0.072810
608,2.232158,1.732158,1.732158,-0.267842,-0.267842,-0.267842,-0.267842,-0.267842,-0.267842,3.732158,...,-0.267842,-0.267842,-0.267842,-0.267842,-0.267842,-0.267842,-0.267842,-0.267842,-0.267842,-0.267842


In [ ]:
#Doing normal SVD
U, sigma, Vt = svds(R_normalized, k = 50) #k is hyperpapameter
sigma = np.diag(sigma) #Σ returned is just the values instead of a diagonal matrix. Convert it to the diagonal matrix form.

In [ ]:
print(R_normalized.keys())

Int64Index([     1,      2,      3,      4,      5,      6,      7,      8,
                 9,     10,
            ...
            193565, 193567, 193571, 193573, 193579, 193581, 193583, 193585,
            193587, 193609],
           dtype='int64', name='movieId', length=9724)


In [ ]:
#also need to add the user means back to get the predicted 5-star ratings
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = matrix.columns)
user_col = [i for i in range(R_normalized.shape[0])]
preds_df.insert(loc=0, column='userId', value=user_col)
preds_df.head()

movieId,userId,1,2,3,4,5,6,7,8,9,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,0,2.167328,0.402751,0.840184,-0.076281,-0.551337,2.504091,-0.890114,-0.026443,0.196974,...,-0.023453,-0.019967,-0.026939,-0.026939,-0.023453,-0.026939,-0.023453,-0.023453,-0.023453,-0.058732
1,1,0.211459,0.006658,0.033455,0.017419,0.183430,-0.062473,0.083037,0.024158,0.049330,...,0.019498,0.016777,0.022219,0.022219,0.019498,0.022219,0.019498,0.019498,0.019498,0.032281
2,2,0.003588,0.030518,0.046393,0.008176,-0.006247,0.107328,-0.012416,0.003779,0.007297,...,0.005909,0.006209,0.005610,0.005610,0.005909,0.005610,0.005909,0.005909,0.005909,0.008004
3,3,2.051549,-0.387104,-0.252199,0.087562,0.130465,0.270210,0.477835,0.040313,0.025858,...,0.004836,0.004172,0.005500,0.005500,0.004836,0.005500,0.004836,0.004836,0.004836,-0.023311
4,4,1.344738,0.778511,0.065749,0.111744,0.273144,0.584426,0.254930,0.128788,-0.085541,...,-0.008042,-0.007419,-0.008664,-0.008664,-0.008042,-0.008664,-0.008042,-0.008042,-0.008042,-0.010127


Making predictions:

In [ ]:
#making recommendation: 
userID = 400
num_recommendations=5
    
# Get and sort the user's predictions
user_row_number = userID - 1 # UserID starts at 1, not 0
sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False)
sorted_user_predictions.head()

movieId
userId    399.000000
318         4.777884
260         4.653620
296         4.470283
2571        4.188326
Name: 399, dtype: float64

In [ ]:
# Get the user's data and merge in the movie information.
user_data = ratings[ratings.userId == (userID)]
print(user_data.head())

       userId  movieId  rating   timestamp
61156     400        6     5.0  1498870480
61157     400       47     5.0  1498870391
61158     400       50     5.0  1498870285
61159     400      260     5.0  1498870148
61160     400      293     5.0  1498870424


In [ ]:
#Top rated movies by user
user_temp= user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId')
user_full = user_temp.sort_values(['rating'], ascending=False)

print(f'User id: {userID} has already rated {user_full.shape[0]} movies.')
print('Top 5 movies:')
user_full.head()

User id: 400 has already rated 43 movies.
Top 5 movies:


,userId,movieId,rating,timestamp,title,genres
0,400,6,5.0,1498870480,Heat (1995),Action|Crime|Thriller
10,400,608,5.0,1498870431,Fargo (1996),Comedy|Crime|Drama|Thriller
1,400,47,5.0,1498870391,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
26,400,3949,5.0,1498870465,Requiem for a Dream (2000),Drama
18,400,1210,5.0,1498870163,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi


In [ ]:
# Recommend the highest predicted rating movies that the user hasn't seen yet.
not_rated = movies[~movies['movieId'].isin(user_full['movieId'])] #removing rated movies
not_rated = not_rated.merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',left_on = 'movieId',right_on = 'movieId') #with predictions
not_rated

,movieId,title,genres,399
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.102185
1,2,Jumanji (1995),Adventure|Children|Fantasy,-0.274635
2,3,Grumpier Old Men (1995),Comedy|Romance,-0.106373
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.069906
4,5,Father of the Bride Part II (1995),Comedy,-0.146367
...,...,...,...,...
9694,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,0.000634
9695,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,0.000984
9696,193585,Flint (2017),Drama,0.000984
9697,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,0.000984


In [ ]:
recommendations = not_rated.rename(columns = {user_row_number: 'Predictions'})
# print(recommendations.head())
recommendations = recommendations.sort_values('Predictions', ascending = False)
# print(recommendations.head())
recommendations = recommendations.iloc[:num_recommendations, :-1]
print(f'Recommending the highest {num_recommendations} predicted ratings movies not already rated.')
print(recommendations.head())

Recommending the highest 5 predicted ratings movies not already rated.
     movieId                                          title             genres
453      527                        Schindler's List (1993)          Drama|War
4108    5952  Lord of the Rings: The Two Towers, The (2002)  Adventure|Fantasy
2121    2858                         American Beauty (1999)      Drama|Romance
1711    2329                      American History X (1998)        Crime|Drama
1480    2028                     Saving Private Ryan (1998)   Action|Drama|War


See recommendation system by Applied AI videos 